In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
# load the data
data = pd.read_csv("../data/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Data Preprocessing

In [4]:
# drop irrelevant columns
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# encode categorial variables
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])

data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
# one-hot encoding of "Geography" column
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])

In [9]:
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [10]:
onehot_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
# combine one-hot encoder columns with the original data
data = pd.concat([data.drop("Geography", axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
# save the encoder
with open("../pkl/label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender, file)

with open("../pkl/onehot_encoder_geo.pkl", "wb") as file:
    pickle.dump(onehot_encoder_geo, file)

In [15]:
# divide the dataset into independent and dependent features
X = data.drop("Exited", axis=1)
y = data["Exited"]

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [17]:
# save the scaler
with open("../pkl/scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

## ANN Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [19]:
# build ANN model
model = Sequential(
    [
        Dense(64, activation="relu", input_shape=(X_train.shape[1],)), # 1st hidden layer, connected with input layer
        Dense(32, activation="relu"), # 2nd hidden layer
        Dense(1, activation="sigmoid") # output layer
    ]
)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# define the optimizer & loss
import tensorflow

optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [22]:
# compile the model (in order to perform forward and backward propagation)
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=["accuracy"]
)

In [40]:
# set up the Tensorboard
log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [41]:
# set up early stopping
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [42]:
# train the model
history = model.fit(X_train, 
                    y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=100, 
                    callbacks=[tensorflow_callback, early_stopping_callback]
                    )

Epoch 1/100
250/250 [==============================] - 0s 813us/step - loss: 0.3147 - accuracy: 0.8689 - val_loss: 0.3533 - val_accuracy: 0.8590
Epoch 2/100
250/250 [==============================] - 0s 536us/step - loss: 0.3149 - accuracy: 0.8708 - val_loss: 0.3543 - val_accuracy: 0.8590
Epoch 3/100
250/250 [==============================] - 0s 541us/step - loss: 0.3128 - accuracy: 0.8719 - val_loss: 0.3581 - val_accuracy: 0.8590
Epoch 4/100
250/250 [==============================] - 0s 536us/step - loss: 0.3119 - accuracy: 0.8710 - val_loss: 0.3544 - val_accuracy: 0.8635
Epoch 5/100
250/250 [==============================] - 0s 567us/step - loss: 0.3085 - accuracy: 0.8717 - val_loss: 0.3711 - val_accuracy: 0.8515
Epoch 6/100
250/250 [==============================] - 0s 535us/step - loss: 0.3104 - accuracy: 0.8704 - val_loss: 0.3662 - val_accuracy: 0.8590
Epoch 7/100
250/250 [==============================] - 0s 581us/step - loss: 0.3048 - accuracy: 0.8750 - val_loss: 0.3662 - val_ac

In [43]:
# save the model
model.save("../model/model.h5")

/Users/abhinandansamal/Projects/Bank_Customer_Churn_Prediction_ANN/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
# load Tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [45]:
%tensorboard --logdir  ../logs/fit # execute twice this line of code

Reusing TensorBoard on port 6006 (pid 51738), started 0:10:45 ago. (Use '!kill 51738' to kill it.)